In [5]:
import sqlite3
import os

In [16]:
folder_path = "spider/database"
select_db = ['musical',
             'farm', 
             'hospital_1', 
             'tvshow', 
             'cinema', 
             'restaurants', 
             'company_employee', 
             'company_1', 
             'company_offic', 
             'singer', 
             'coffee_shop']

db = []

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files = os.listdir(folder_path)
    for file in files:
        if file in select_db:
            db_path = os.path.join(folder_path, file)
            sqlite_db = [os.path.join(db_path, sql) for sql in os.listdir(db_path) if ".sqlite" in sql]
            db.append(*sqlite_db)

In [17]:
db

['spider/database/musical/musical.sqlite',
 'spider/database/farm/farm.sqlite',
 'spider/database/hospital_1/hospital_1.sqlite',
 'spider/database/tvshow/tvshow.sqlite',
 'spider/database/cinema/cinema.sqlite',
 'spider/database/restaurants/restaurants.sqlite',
 'spider/database/company_employee/company_employee.sqlite',
 'spider/database/company_1/company_1.sqlite',
 'spider/database/coffee_shop/coffee_shop.sqlite',
 'spider/database/singer/singer.sqlite']

In [33]:
def get_schema(sqlite_db):
    connection = sqlite3.connect(sqlite_db)
    cursor = connection.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        print(f"Table: {table_name}")

        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        for column in columns:
            column_name = column[1]
            print(f"  Column: {column_name}")

        print()
    
    cursor.close()
    connection.close()

In [38]:
# for table in db:
#     get_schema(table)
#     print('---------------------------------')